In [1]:
from collections import OrderedDict
from itertools import product
from pyCBT.data.providers.mt.pairs import get_mt_pairs
from highcharts import Highchart

import ffn

np = ffn.core.np
pd = ffn.data.pd
plt = ffn.core.plt

In [2]:
# define constants
# time frame in periods/minutes (because data is sampled in minutes)
TIME_FRAME = 30
# define target price
PRICE = "CLOSE"
# define volume column
VOLUME = "VOLUME"

pairs = get_mt_pairs()

PAIRS = pairs.keys()[:10]

# extract price for each instrument
price = pd.DataFrame(index=range(5*TIME_FRAME))
for pair_name in PAIRS:
    price[pair_name] = pairs[pair_name].filter(like=PRICE).values[-5*TIME_FRAME:]

# compute correlations
C = pd.DataFrame(data=np.nan, index=PAIRS, columns=PAIRS)
for i, j in product(range(len(PAIRS)), range(len(PAIRS))):
    price_i, price_j = price[PAIRS[i]][-TIME_FRAME:].values, price[PAIRS[j]][-TIME_FRAME:].values
    C.loc[PAIRS[i]][PAIRS[j]] = np.corrcoef(price_i, price_j)[1, 0]

In [4]:
chart_options = {
    'chart': {
        'type': 'heatmap',
        'marginTop': 40,
        'marginBottom': 80,
        'plotBorderWidth': 1
    },
    'title': {
        'text': 'Correlations'
    },

    'xAxis': {
        'categories': PAIRS
    },

    'yAxis': {
        'categories': PAIRS,
        'title': {
            'text': None
        }
    },
    'colorAxis': {
        'min': -1,
        'max': +1,
#         'minColor': '#86BAED',
        'maxColor': '#F46363',
#         'stops': [[-1, '#86BAED'], [0, '#FFFFFF'], [+1, '#F46363']]
    },
    'legend': {
        'align': 'right',
        'layout': 'vertical',
        'margin': 0,
        'verticalAlign': 'top',
        'y': 25,
        'symbolHeight': 280
    },
    'tooltip': {
        'formatter': "function () {\
            return '<b>' + this.series.xAxis.categories[this.point.x] + '</b> correlates <br><b>' +\
                this.point.value + '</b> with <br><b>' + this.series.yAxis.categories[this.point.y] + '</b>';\
        }"
    }
}

chart_dataset = [[i, j, round(C.iloc[i][j],2) if i>j else None] for i, j in product(range(len(PAIRS)), range(len(PAIRS)))]

chart = Highchart()
chart.set_dict_options(chart_options)
chart.add_data_set(chart_dataset, 'heatmap', dataLabels={'enabled': True, 'color': '#000000'})
chart